In [1]:
import sqlite3
sqlite_file = 'database.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
#c.execute('''CREATE TABLE Table2 (
#            "file_name" TEXT, 
#            "definition" TEXT
#            )''')

In [2]:
import glob
doc_list=glob.glob("*.pdf")
from PyPDF2 import PdfFileReader
import re
def parse_pdf(file_name):
    pdfReader=PdfFileReader(file_name)
    number_of_pages=pdfReader.numPages
    pattern='(Section [0-9]+\.[0-9]+\.*[^\.]+)[\.| ]+([0-9]+)'
    text=''
    for page in range(50):
        text+=(pdfReader.getPage(page).extractText().replace('\n',''))
    content=re.findall(pattern,text)
    toc={}
    for x in content:
        section=x[0]
        pagenum=x[1]
        if section not in toc:
            toc[section]=pagenum
    searchword='Section 1.1'
    pages_text=[]
    words_start_pos={}
    words={}
    pn=[] 
    for page in range(50):
        pages_text.append(pdfReader.getPage(page).extractText())
        words_start_pos[page]=[dwg.start() for dwg in re.finditer(searchword, pages_text[page])]
        words[page]=[pages_text[page][value:value+len(searchword)] for value in words_start_pos[page]]
    for page in words:
        for i in range(0,len(words[page])):
            if str(words[page][i]) != 'nan':
                pn.append(page)
    stopword=r'TABLE OF CONTENTS|Table of Contents'
    for num in pn:
        text=pdfReader.getPage(num).extractText()
        if re.search(stopword,text)== None:
            def_pagenum=num
    diff=def_pagenum-int(list(toc.values())[0])
    for key in toc.keys():
        toc[key]=int(toc[key])+diff
    keyword='Definitions'
    for i in range(len(toc)):
        section=list(toc)[i]
        if re.search(keyword,section)!=None:
            startpage=list(toc.values())[i]
            endpage=list(toc.values())[i+1]
    targettext=''
    for page in range(startpage,endpage+1):
        targettext+=(pdfReader.getPage(page).extractText())
    definition={}
    pattern=r'\"([^\"]+)\": ([^\"]+)\. '
    defs=re.findall(pattern, targettext.replace('\n',''))
    for x in defs:
        term=x[0]
        explanation=x[1]
        if term not in definition:
            definition[term]=explanation
    return definition

In [3]:
file_name="Octagon.pdf"
definition=parse_pdf(file_name)

FileNotFoundError: [Errno 2] No such file or directory: 'Octagon.pdf'

In [11]:
for term,explanation in definition.items():
    c.execute("INSERT INTO Table2 VALUES (?,?)", (term, explanation))

In [22]:
c.execute("SELECT * FROM Table2")

In [23]:
c.fetchall()

[('17g-5 Information', ' The meaning specified in Section 14.16'),
 ('17g-5 Website',
  ' A password-protected internet website which shall initially be located at https://www.structuredfn.com'),
 ('25% Limitation',
  ' A limitation that is exceeded only if Benefit Plan Investors hold 25% or more of the value of any class of equity interests in the Issuer, as calculated under the Plan Asset Regulations'),
 ('Accepted Purchase Request', ' The meaning specified in Section 9.9(c)'),
 ("Accountants' Certificate",
  " A certificate of the firm or firms appointed by the Issuer pursuant to Section 10.9(a).  For the avoidance of doubt, notwithstanding anything to the contrary set forth herein, no Accountants' Certificate shall be provided to or otherwise shared with any Rating Agency"),
 ("Accountants' Effective Date Comparison AUP Report",
  ' The meaning specified in Section 7.17'),
 ("Accountants' Effective Date Recalculation AUP Report",
  ' The meaning specified in Section 7.17'),
 ('Acco

In [85]:
values_string

"( The meaning specified in Section 14.16, A password-protected internet website which shall initially be located at https://www.structuredfn.com, A limitation that is exceeded only if Benefit Plan Investors hold 25% or more of the value of any class of equity interests in the Issuer, as calculated under the Plan Asset Regulations, The meaning specified in Section 9.9(c), A certificate of the firm or firms appointed by the Issuer pursuant to Section 10.9(a).  For the avoidance of doubt, notwithstanding anything to the contrary set forth herein, no Accountants' Certificate shall be provided to or otherwise shared with any Rating Agency, The meaning specified in Section 7.17, The meaning specified in Section 7.17, Each of (i) the Payment Account, (ii) the Collection Account, (iii) the Ramp-Up Account, (iv) the Expense Reserve Account, (v) the Interest Reserve Account, (vi) the Custodial Account, (vii) the Unfunded Exposure Account, (viii) the Contribution Account and (ix) each Hedge Coun

In [31]:
c.fetchall()

[]

In [19]:
definition

{'17g-5 Information': ' The meaning specified in Section 14.16',
 '17g-5 Website': ' A password-protected internet website which shall initially be located at https://www.structuredfn.com',
 '25% Limitation': ' A limitation that is exceeded only if Benefit Plan Investors hold 25% or more of the value of any class of equity interests in the Issuer, as calculated under the Plan Asset Regulations',
 'Accepted Purchase Request': ' The meaning specified in Section 9.9(c)',
 "Accountants' Certificate": " A certificate of the firm or firms appointed by the Issuer pursuant to Section 10.9(a).  For the avoidance of doubt, notwithstanding anything to the contrary set forth herein, no Accountants' Certificate shall be provided to or otherwise shared with any Rating Agency",
 "Accountants' Effective Date Comparison AUP Report": ' The meaning specified in Section 7.17',
 "Accountants' Effective Date Recalculation AUP Report": ' The meaning specified in Section 7.17',
 'Accounts': ' Each of (i) the 

In [46]:
conn.commit()
conn.close()